# 量子・古典ハイブリッドアルゴリズムとしての量子機械学習を使って、新しい素粒子現象の発見を目指す

この実習では、量子・古典ハイブリッドアルゴリズムの応用である**量子機械学習**の基本的な実装を学んだのち、その活用例として**素粒子実験での新粒子探索への応用**を考えます。別の実習で学んだ変分量子固有値ソルバー法（VQE）も量子・古典ハイブリッドアルゴリズムの一種です。ここでは、特に量子コンピュータを機械学習に応用することで機械学習の性能を上げるという観点から提案された、**変分量子回路の学習**について学びます。

## 内容
1. [はじめに](#introduction)
2. [探索する新現象](#model)
3. [量子機械学習](#procedure)
    1. [学習データの準備](#data)
    2. [量子状態の生成](#state_preparation)
    3. [変分フォームを使った状態変換](#variational_form)
    4. [測定とモデル出力](#measurement)
    5. [パラメータの最適化](#parameter_optimization)
4. [参考文献](#references)


## はじめに <a id='introduction'></a>

近年、機械学習の分野においてディープラーニングが注目を浴びています。ディープラーニングは**ニューラルネットワーク**の隠れ層を多層にすることで、入力と出力の間の複雑な関係を学習することができます。その学習結果を使って、新しい入力データに対して出力を予測することが可能になります。量子機械学習アルゴリズムは、このニューラルネットワークの部分を変分量子回路に置き換えたもので構成されます。つまり、ニューラルネットワークでの各ニューロン層への重みを調節する代わりに、変分量子回路のパラメータ（例えば回転ゲートの回転角）を調整することで入力と出力の関係を学習しようという試みです。
この仕組みを使うことで、量子力学の重ね合わせの原理から**指数関数的に多数の計算基底**で量子状態を表現できる可能性があり、そこに量子機械学習の強みがあると期待されています。

多項式で与えられる数の量子ゲートを使って、指数関数的に増える関数を表現できる可能性があるところに量子機械学習の強みがありますが、誤り訂正機能を持たない中規模の量子コンピュータ (*Noisy Intermediate-Scale Quantum*デバイス, 略してNISQ）で量子加速を実現できるかどうかには確証はありません。しかしNISQデバイスでの動作に適したアルゴリズムであるため、2019年3月にはIBMの実験チームによる実機での実装がすでに行われ、結果も論文 [[2]](https://www.nature.com/articles/s41586-019-0980-2)として出版されています。


## 探索する新現象<a id='model'></a>

この実習では、素粒子現象の基本モデル（標準模型と呼ばれる）を超える新しい素粒子現象として、超対称性理論（Supersymmetry、略してSUSY）から予言される新粒子の探索を考えます。下の図はSUSY信号（左側）と標準模型でのバックグラウンド（右側）のファインマンダイアグラムを表していて、この二つの物理過程を量子機械学習で分類することを試みます。

<img src="figs/susy_bg.png" width=70%>

(図の引用：参考文献 [[3]](https://www.nature.com/articles/ncomms5308))

## 量子機械学習<a id='procedure'></a>

分類（classification）を実行する量子機械学習アルゴリズムは、一般的には以下のような順番で実行されます。

1. **学習データ**$\{(\boldsymbol{x}_i, y_i)\}$を用意する。$\boldsymbol{x}_i$は入力データのベクトル、$y_i$は入力データに対する正解のラベル（教師データ）とする（$i$は学習データのサンプルを表す添字）。
2. 入力$\boldsymbol{x}$から何らかの規則で決まる回路$U_{\text{in}}(\boldsymbol{x})$（**特徴量マップ**と呼ぶ）を用意し、$\boldsymbol{x}_i$の情報を埋め込んだ入力状態$|\psi_{\rm in}(\boldsymbol{x}_i)\rangle = U_{\text{in}}(\boldsymbol{x}_i)|0\rangle$を作る。
3. 入力状態にパラメータ$\theta$に依存したゲート$U(\theta)$（**変分フォーム**と呼ぶ）を掛けたものを出力状態$|\psi_{\rm out}(\boldsymbol{x}_i, \theta)\rangle = U(\theta)|\psi_{\rm in}(\boldsymbol{x}_i)\rangle$とする
4. 出力状態のもとで何らかの**観測量**を測定し、測定値$O_i$を得る。例えば、最初の量子ビットで測定したパウリ$Z$の期待値$\langle Z_1\rangle = \langle \psi_{\rm out} |Z_1|\psi_{\rm out} \rangle$。
5. $F$を適当な関数として、$F(O_i)$をモデルの出力$y(\boldsymbol{x}_i, \theta)$とする。
6. 正解ラベル$y_i$とモデル出力$y(\boldsymbol{x}_i, \theta)$の間の乖離を表す**コスト関数**$L(\theta)$を定義し、古典計算でコスト関数を計算する。
7. 2-6のプロセスを繰り返すことで、コスト関数を最小化する$\theta=\theta^*$を求める。
8. $y(\boldsymbol{x}, \theta^*)$が求める**予測モデル**になる。

まず、必要なライブラリを最初にインポートします。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.circuit.library import TwoLocal, ZFeatureMap, ZZFeatureMap
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

import logging
from qiskit.aqua import set_qiskit_aqua_logging
set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

### 学習データの準備<a id='data'></a>


In [ ]:
training_size = 20
testing_size = 20
niter = 20
backend_name = 'ibmq_montreal'
random_seed = 10598

df = pd.read_csv("data_files/SUSY_1K.csv",
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

if feature_dim == 3:
    SelectedFeatures = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    SelectedFeatures = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    SelectedFeatures = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

df_sig = df.loc[df.isSignal==1, SelectedFeatures]
df_bkg = df.loc[df.isSignal==0, SelectedFeatures]

df_sig_training = df_sig.values[:training_size]
df_bkg_training = df_bkg.values[:training_size]
df_sig_test = df_sig.values[training_size:training_size+testing_size]
df_bkg_test = df_bkg.values[training_size:training_size+testing_size]
training_input = {'1':df_sig_training, '0':df_bkg_training}
test_input = {'1':df_sig_test, '0':df_bkg_test}
#print('train_input =',training_input)
#print('test_input =',test_input)

datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
datapoints_tr, class_to_label_tr = split_dataset_to_data_and_labels(training_input)

### 量子状態の生成<a id='state_preparation'></a>

まず、入力値$\boldsymbol{x}_i$を初期状態$|0\rangle^{\otimes n}$に埋め込むためのゲート $U_{\rm in}(\boldsymbol{x}_i)$を作成する。まず、参考文献[2]に従い$U_{\phi_{\{k\}}}(\boldsymbol{x}_i)=\exp(i\phi_{\{k\}}(\boldsymbol{x}_i)Z_k)$（$k$は入力値$\boldsymbol{x}_i$のベクトル要素の添字）とする。ここで$\phi_{\{k\}}(\boldsymbol{x}_i)=x_i^k$と決めて（$x_i^k$は$\boldsymbol{x}_i$の$k$番目要素）、入力値$\boldsymbol{x}_i$を$k$個の量子ビットに埋め込む。この$U_{\phi_{\{k\}}}(x)$にアダマール演算子を組み合わせることで、全体として

$$
U_{\rm in}(\boldsymbol{x}_i) = U_{\phi}(\boldsymbol{x}_i) H^{\otimes n}, \:U_{\phi}(\boldsymbol{x}_i) = \exp\left(i \sum_{k=1}^n \phi_{\{k\}}(\boldsymbol{x}_i)Z_k\right)
$$

が得られる。この$U_{\rm in}(\boldsymbol{x}_i)$をゼロの標準状態に適用することで、入力値$\boldsymbol{x}_i$は$|\psi_{\rm in}(\boldsymbol{x}_i)\rangle =U_{\rm in}(\boldsymbol{x}_i)|0\rangle^{\otimes n}$という量子状態に変換されることになる。

In [ ]:
feature_map = ZFeatureMap(feature_dim, reps=1)

### 変分フォームを使った状態変換<a id='variational_form'></a>

#### 変分量子回路$U(\theta)$の構成
次に、最適化すべき変分量子回路$U(\theta)$を作っていく。これは以下の３手順で行う。

1. 2量子ビットゲートの作成（エンタングルメント）
2. 回転ゲートの作成
3. 1.と2.のゲートを交互に組み合わせ、1つの大きな変分量子回路$U(\theta)$を作る

##### 2量子ビットゲートの作成
エンタングルメントを活用してモデルの表現能力を上げるため、ここではControlled-$Z$ゲート（$CZ$）を使う。

##### 回転ゲートと$U(\theta)$の作成
$CZ$ゲートによるエンタングルメント生成$U_{\text{ent}}$と、$j \:(=1,2,\cdots n)$番目の量子ビットに回転ゲート

$$
U_{\text{rot}}(\theta_j^{(i)}) = R_j^Y(\theta_{j1}^{(i)})R_j^Z(\theta_{j2}^{(i)})
$$

を掛けたものを組み合わせて変分量子回路$U(\theta)$を構成する。ここで、$i$は量子回路の層を表しており、$U_{\text{ent}}$と上記の回転を合計$d$層繰り返す。実際は、最初に回転ゲート$U_{\text{rot}}$を一度適用してから$d$層繰り返す構造を使うため、全体としては

$$
 U \left( \{ \theta_j^{(i)} \}_{i,j} \right) = \prod_{i=1}^d \left( \left( \prod_{j=1}^n U_{\text{rot}}(\theta_j^{(i)})\right) \cdot U_{\text{rand}} \right) \cdot \prod_{i=1}^d U_{\text{rot}}(\theta_j^{(0)})
$$

という形式の変分量子回路を用いる。パラメータとしては、全部で$2n(d+1)$個のパラメータがあることになる。各$\theta$の初期値は$[0, 2\pi]$の一様分布にとっておく。

In [ ]:
two = TwoLocal(feature_dim, ['ry','rz'], 'cz', 'full', reps=1)
print(two)

### 測定とモデル出力<a id='measurement'></a>

今回は、0番目の量子ビットのpauli Zの、出力状態$|\psi_{\rm out}\rangle$での期待値をモデルの出力とする。
すなわち、$y(\theta, x_i) = \langle Z_0 \rangle = \langle \psi_{\rm out}|Z_0|\psi_{\rm out}\rangle$である。

In [ ]:
result = vqc.run(quantum_instance)
print(" --- Testing success ratio: ", result['testing_accuracy'])

### パラメータの最適化<a id='parameter_optimization'></a>

コスト関数 $L(\theta)$は、教師データと予測データの平均二乗誤差(MSE)とする。

In [ ]:
optimizer = COBYLA(maxiter=niter, disp=True)

vqc = VQC(optimizer, feature_map, two, training_input, test_input)
quantum_instance = QuantumInstance(backend=backend, shots=1024,
                                   seed_simulator=random_seed, seed_transpiler=random_seed,
                                   skip_qobj_validation=True)

result = vqc.run(quantum_instance)
print(" --- Testing success ratio: ", result['testing_accuracy'])



## 参考文献<a id='references'></a>
1. K. Mitarai, M. Negoro, M. Kitagawa, and K. Fujii, “Quantum circuit learning”, [Phys. Rev. A 98, 032309 (2018)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.032309)
2. V. Havlicek _et al._ , “Supervised learning with quantum-enhanced feature spaces”, [Nature 567, 209–212 (2019)](https://www.nature.com/articles/s41586-019-0980-2)
3. P. Baldi, P. Sadowski, and D. Whiteson, “Searching for exotic particles in high-energy physics with deep learning”, [Nature Commun. 5, 4308 (2014)](https://www.nature.com/articles/ncomms5308)